In [1]:
from sage.all import * 
import struct

DR = RealField(53)

DD = RealField(190)

def double_to_hex(f):
    packed = struct.pack('>d', float(f))
    return '0x' + packed.hex()

def split_double_double(x):
    # Split RR value x into hi + lo (double-double)
    x = RealField(190)(x).exact_rational()
    x_hi = DR(x)  # convert to f64
    x_lo = x - DD(x_hi)
    return (x_lo,x_hi)

def split_triple_double(x):
    # Split RR value x into hi + lo (double-double)
    x_hi = DR(x)  # convert to f64
    x_mid = DR(x - DD(x_hi))
    x_lo = x - DD(x_hi) - DD(x_mid)
    return (x_lo, x_mid, x_hi)

def print_double_double(mark, x):
    splat = split_double_double(x)
    print(f"{mark}({double_to_hex(splat[0])}, {double_to_hex(splat[1])}),")

def print_triple_double(mark, x):
    splat = split_triple_double(x)
    print(f"{mark}({double_to_hex(splat[0])}, {double_to_hex(splat[1])}, {double_to_hex(splat[2])}),")

In [58]:
from sage.all import *
from mpmath import mp, besseli, taylor

mp.prec = 500

def refined_approx(x):
    if x == 0:
        return 0
    p1 = besseli(1, x) / x * 2 - mp.mpf(1)
    x_over_two = mp.mpf(x) / 2
    x_over_two_sqr = x_over_two * x_over_two
    p2 = x_over_two_sqr * mp.mpf(0.5)
    p1 = p1 - p2
    x_over_two_p4 = x_over_two_sqr * x_over_two_sqr
    p1 = p1
    return p1

terms = 70

coeffs = taylor(lambda x: refined_approx(x), mp.mpf('0'), terms)

print(refined_approx(5))
print(coeffs)

# Step 3: Build series in terms of y = (x/2)^2
R = PolynomialRing(RealField(450), 'y')
y = R.gen()
f = R(0)

for n in range(2, terms, 2):
    k = n // 2
    c = RealField(450)(coeffs[n])
    if n >= 1:
        f += R(c) * y**(k-1) * (4**k)
    else:
        f += R(c) * y**(k-1) * (4**k)

print(f)

rat_approx = SR(f).power_series(RealField(100)).pade(16, 0)
numer = rat_approx.numerator()
print(numer)

5.60925685698021087965722018070400338422594974731955926336123209216158518748673667144688016736989945676635607653560767434477873457206268940519083401459
[mpf('0.0'), mpf('0.0'), mpf('0.0'), mpf('0.0'), mpf('0.00520833333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333413'), mpf('0.0'), mpf('0.000108506944444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444461'), mpf('0.0'), mpf('0.00000135633680555555555555555555555555555555555555555555555555555555555555555555555555555555555555555555555555555555555555555555555555555555555555555555555553'), mpf('0.0'), mpf('0.0000000113028067129629629629629629629629629629629629629629629629629629629629629629629629629629629629629629629629629629629629629629629629629629629629629629629646'), mpf('0.0'), mpf('0.00000000006727861138668430335097001763668430335097001763668430335097001

In [57]:
values = [
' 1 ',
' 0.25 ',
' 2.777777777777777623580135468728258274495601654052734375e-2 ',
' 1.73611111111111708626975058677999186329543590545654296875e-3 ',
' 6.9444444444434957615201742253674410676467232406139373779296875e-5 ',
' 1.92901234568737175921969949354828344212364754639565944671630859375e-6 ',
' 3.936759888676649827268097590719786449398043259861879050731658935546875e-8 ',
' 6.1511873441828956828506549147955913270369165957163204438984394073486328125e-10 ',
' 7.59405796954071993550940007343340288313571218026254427968524396419525146484375e-12 ',
' 7.5940671293210007585286363412648594930598293373247287263438920490443706512451171875e-14 ',
' 6.275961133446377019580129016152441639440058107450581825759172716061584651470184326171875e-16 ',
' 4.35959955051298758216272254397780728544635283250481549988109009063919074833393096923828125e-18 ',
' 2.57016360879182038857041181827969219806917743324416265253244684441114031869801692664623260498046875e-20 ',
' 1.3602568196404245484903285697695559574915073549807523587646469927225911789037127164192497730255126953125e-22 ',
' 4.3519323177819898788562312469719088894011000991151465841162815470673230766607275654678232967853546142578125e-25 ',
' 5.26017447655109689551220236016711308401702926604489694949032193847215910752213030576740493415854871273040771484375e-27 ',
' -1.777282201721818351530714197390393454320429252790767397879131152377718644796121549855882904012105427682399749755859375e-29 ',

]

for i, c in enumerate(values):
    c = RealField(1000)(c.replace(" ", ""))
    # print_double_double("", c)
    # print(RealField(53)(c))
    print("f64::from_bits(" + double_to_hex(c) + "),")

f64::from_bits(0x3ff0000000000000),
f64::from_bits(0x3fd0000000000000),
f64::from_bits(0x3f9c71c71c71c71c),
f64::from_bits(0x3f5c71c71c71c738),
f64::from_bits(0x3f123456789aba23),
f64::from_bits(0x3ec02e85c089d88b),
f64::from_bits(0x3e6522a43f5a944f),
f64::from_bits(0x3e0522a440661101),
f64::from_bits(0x3da0b31317b0a80b),
f64::from_bits(0x3d35601a20cd3aa3),
f64::from_bits(0x3cc69c8d6cdb26cf),
f64::from_bits(0x3c541ae81d6d4a08),
f64::from_bits(0x3bde57d7c636c9c3),
f64::from_bits(0x3b648e3d1ac6be91),
f64::from_bits(0x3ae0d5f1ec46f644),
f64::from_bits(0x3a7a0c10368f1ea7),
f64::from_bits(0xb9f6879c496e231c),


In [53]:
from sage.all import *
from mpmath import mp, besseli

# Set precision
mp.dps = 50  # digits of precision
R = RealField(100)

# Max float32 value
FLT_MAX = R('3.4028235e38')

# Wrapper: compute I1(x) using mpmath and compare to FLT_MAX
def f(x):
    x_mp = mp.mpf(x)
    return float(besseli(1, x_mp)) - float(FLT_MAX)

# Use Sage's root finder
x0 = find_root(f, 80, 100)
print(f"Overflow threshold for I1(x) in float32: x ≈ {x0}")



Overflow threshold for I1(x) in float32: x ≈ 91.90626485018107
